In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
from sklearn.metrics import mean_squared_error,classification_report,f1_score,confusion_matrix,accuracy_score

import xgboost as xgb
import lightgbm as lgb

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate, learning_curve,train_test_split,GridSearchCV, cross_val_score, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier

from sklearn.preprocessing import LabelEncoder

from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.drop("id",axis=1,inplace=True)
df.head(2)

In [ ]:
df.gender.value_counts()

In [ ]:
df.drop(df[df["gender"]=="Other"].index,axis=0,inplace=True)

In [ ]:
df.gender.value_counts()

In [ ]:
sns.distplot(df.age);

In [ ]:
sns.distplot(df.bmi);

In [ ]:
sns.distplot(df.avg_glucose_level);

In [ ]:
df.head()

In [ ]:
df.hypertension.value_counts()

In [ ]:
df.heart_disease.value_counts()

In [ ]:
df.ever_married.value_counts()

In [ ]:
df.work_type.value_counts()

In [ ]:
df.Residence_type.value_counts()

In [ ]:
df.smoking_status.value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df.stroke.value_counts()

In [ ]:
df.bmi.mean()

In [ ]:
df.bmi.fillna(df.bmi.mean(),inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.head(2)

In [ ]:
le = LabelEncoder()
df["gender"] = le.fit_transform(df["gender"])
df["ever_married"] = le.fit_transform(df["ever_married"])
df["work_type"] = le.fit_transform(df["work_type"])
df["Residence_type"] = le.fit_transform(df["Residence_type"])
df["smoking_status"] = le.fit_transform(df["smoking_status"])

In [ ]:
df.head()

# Model

In [ ]:
log_model = LogisticRegression()

gbc_model = GradientBoostingClassifier(random_state=14)

rfc_model = RandomForestClassifier(criterion='gini', n_estimators=999,max_depth=4, random_state=14)

lgb_model = lgb.LGBMClassifier(num_iterations=550, learning_rate=0.01055,max_depth=3, random_state=14)

xgb_model = xgb.XGBClassifier(objective="binary:hinge")

gnb_model = GaussianNB()

mlpc_model = MLPClassifier(random_state=14)

svc_model = SVC(probability=True)

knn_model = KNeighborsClassifier(n_neighbors=19,leaf_size=20)

In [ ]:
models = [log_model, gbc_model, rfc_model, lgb_model, xgb_model, gnb_model, mlpc_model, svc_model,knn_model]
model_names = ["LogisticRegression","GradientBoostingClassifier","RandomForestClassifier","LGBMClassifier","XGBClassifier","GaussianNB","MLPClassifier","SVC","KNeighborsClassifier"]

x = df.drop('stroke', axis=1)
y = df['stroke']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=42)

In [ ]:
print("TRAIN SCORE\n\n")
print("-----------------------------------------")
for i in range(len(models)):
    model = models[i].fit(x_train,y_train)
    print(model_names[i],"Score = %",model.score(x_train,y_train)*100)

In [ ]:
print("TEST SCORE\n\n")
print("-----------------------------------------")
for i in range(len(models)):
    model = models[i].fit(x_train,y_train)
    print(model_names[i],"Score = %",model.score(x_test,y_test)*100)

# LGBMClassifier

In [ ]:
light_params = {"n_estimators":[100,500,1000,2000],
         "subsample":[0.6,0.8,1.0],
         "max_depth":[3,4,5,6,7],
         "learning_rate":[0.1,0.01,0.02,0.05],
         "min_child_samples":[2,5,10,20]}

In [ ]:
random_search = RandomizedSearchCV(lgb_model, light_params, cv=10, random_state=1, n_jobs=-1, verbose=2 ).fit(x_train,y_train)
random_search.best_params_

In [ ]:
light_tuned = lgb.LGBMClassifier(n_estimators = 100,
                         subsample= 0.8, 
                         max_depth=3 , 
                         learning_rate=0.01, 
                         min_child_samples= 2).fit(x_train,y_train)
print("LGBMClassifier","Score = %",light_tuned.score(x_test,y_test))
print("LGBMClassifier","F1 Score = %",f1_score(y_test,light_tuned.predict(x_test))*100)

# KNeighborsClassifier

In [ ]:
knn_params = {"n_neighbors":np.arange(1,50),
         "leaf_size":[20,30,50,100,1000,2000]}

In [ ]:
random_search = RandomizedSearchCV(knn_model, knn_params, cv=10, random_state=1, n_jobs=-1, verbose=2 ).fit(x_train,y_train)
random_search.best_params_

In [ ]:
knn_tuned = KNeighborsClassifier(n_neighbors = 24, 
                                 leaf_size= 2000).fit(x_train,y_train)
print("KNeighborsClassifier","Score = %",knn_tuned.score(x_test,y_test))
print("KNeighborsClassifier","F1 Score = %",f1_score(y_test,knn_tuned.predict(x_test))*100)

# MLPClassifier

In [ ]:
mlpc_params = {"alpha":[0.1,0.2,0.02,0.01,0.005,0.0001,0.00001],
         "hidden_layer_sizes":[(10,20,30),(10,10,10),(100,100,100),(100,100),(3,5),(5,3),(10,10)],
         "solver":["lbfgs","sgd","adam"],
         "activation":["relu","logistic"]}

In [ ]:
random_search = RandomizedSearchCV(mlpc_model, mlpc_params, cv=10, random_state=1, n_jobs=-1, verbose=2 ).fit(x_train,y_train)
random_search.best_params_

In [ ]:
mlpc_tuned = MLPClassifier(alpha = 0.0001, 
                          hidden_layer_sizes= (5,3),
                          solver="sgd" ,
                          activation= "logistic").fit(x_train,y_train)
print("MLPClassifier","Score = %",mlpc_tuned.score(x_test,y_test))
print("MLPClassifier","F1 Score = %",f1_score(y_test,mlpc_tuned.predict(x_test))*100)

# RandomForestClassifier

In [ ]:
r_forest_params = {"max_depth":[2,3,5,8,10,20],
         "max_features":[2,5,8],
         "n_estimators":[10,100,500,1000,2000],
         "min_samples_split":[2,5,10,20]}

In [ ]:
random_search = RandomizedSearchCV(rfc_model, r_forest_params, cv=10, random_state=1, n_jobs=-1, verbose=2 ).fit(x_train,y_train)
random_search.best_params_

In [ ]:
rfc_tuned = RandomForestClassifier(n_estimators = 100, 
                          max_depth= 20,
                          max_features= 2,
                          min_samples_split= 2).fit(x_train,y_train)
print("RandomForestClassifier","Score = %",rfc_tuned.score(x_test,y_test))
print("RandomForestClassifier","F1 Score = %",f1_score(y_test,rfc_tuned.predict(x_test))*100)

## --------------------------------------------------------------------------------------------------------------------------------

In [ ]:
print("Tuned Scores\n\n")
print("-----------------------------------------")
print("LGBMClassifier","Score = %",light_tuned.score(x_test,y_test))
print("KNeighborsClassifier","Score = %",knn_tuned.score(x_test,y_test))
print("MLPClassifier","Score = %",mlpc_tuned.score(x_test,y_test))
print("RandomForestClassifier","Score = %",rfc_tuned.score(x_test,y_test))

In [ ]:
print("Tuned F1 Scores\n\n")
print("-----------------------------------------")
print("LGBMClassifier","F1 Score = %",f1_score(y_test,light_tuned.predict(x_test))*100)
print("KNeighborsClassifier","F1 Score = %",f1_score(y_test,knn_tuned.predict(x_test))*100)
print("MLPClassifier","F1 Score = %",f1_score(y_test,mlpc_tuned.predict(x_test))*100)
print("RandomForestClassifier","F1 Score = %",f1_score(y_test,rfc_tuned.predict(x_test))*100)